In [1]:


import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
# import lightgbm as lgb
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [3]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [4]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [5]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Scale the data
scalar = MinMaxScaler()
X = scalar.fit_transform(X)
X_testdata = scalar.transform(X_testdata)

# Apply PCA
# pca = PCA(n_components=20)  # Retain 95% of the variance
# X = pca.fit_transform(X)
# X_testdata = pca.transform(X_testdata)

In [7]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Initialize KNN Classifier with optimized parameters
knn = KNeighborsClassifier(
    n_neighbors=5,       # Increase the number of neighbors to use
    weights='distance',   # Weight points by the inverse of their distance
    algorithm='auto',     # Algorithm to compute the nearest neighbors
    leaf_size=20,         # Decrease leaf size for BallTree or KDTree
    p=1,                  # Use Manhattan distance (L1 norm)
    n_jobs=-1             # Use all available cores
)

knn.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Predict the probabilities for the test set
y_prob = knn.predict_proba(X_test)[:, 1]

# Evaluate the AUC score of the classifier
auc = roc_auc_score(y_test, y_prob)
print(f"AUC Score: {auc:.2f}")

Accuracy: 1.00
AUC Score: 0.55


In [9]:
# # Get feature importances
# feature_importances = clf.feature_importances_

# # Create a DataFrame for feature importances
# feature_importances_df = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': feature_importances
# })

# # Sort the DataFrame by importance
# feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# # Print the feature importances
# print(feature_importances_df)

# top_features = feature_importances_df['Feature'].head(45).tolist()

In [10]:
# X_train_top = X_train[top_features]
# X_test_top = X_test[top_features]
# X_testdata_top = X_testdata[top_features]

# # Re-train the model with the top features
# clf.fit(X_train_top, y_train)

In [11]:
knn.fit(X, y)

KNeighborsClassifier(leaf_size=20, n_jobs=-1, p=1, weights='distance')

In [12]:
# Predict probabilities for the test set
y_pred_proba_testdata = knn.predict_proba(X_testdata)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[0.         0.1429095  0.10377412 0.1450159  0.191526   0.18178828
 0.20242189 0.20260749 0.06035764 0.18423939 0.13883167 0.13065585
 0.18913208 0.18510061 0.17620153 0.20405237 0.19188206 0.19004335
 0.13151494 0.09523171 0.18912173 0.20455939 0.20128701 0.17702158
 0.18934943 0.12679847 0.06908686 0.18743729 0.17257826 0.15541784
 0.11412572 0.21470446 0.17785126 0.0744865  0.18991432 0.05885023
 0.14690793 0.15770163 0.18106444 0.18593071 0.09269635 0.18617573
 0.20354159 0.12658812 0.40607137 0.07971008 0.20005354 0.06029575
 0.06978441 0.10741536 0.07866839 0.18582024 0.19918179 0.10729177
 0.15110542 0.14179871 0.09822814 0.19374416 0.21914155 0.06451363
 0.16222055 0.1716203  0.17677839 0.19087027 0.13624175 0.20476757
 0.10733355 0.19481785 0.14637413 0.1714907  0.04459813 0.08732466
 0.18601419 0.16369775 0.19991545 0.19391719 0.19899299 0.20645049
 0.18453856 0.17139441 0.19396112 0.15816542 0.19441771 0.09987028
 0.1845